<a href="https://colab.research.google.com/github/Ali-MH-Mansour/Keyword-Extraction/blob/main/parsers_spacy_allen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# # SemEval2010 wiki20
# import os
# path = "/content/drive/MyDrive/PHD/Data/Inspec/"

# all_files = os.listdir(path+"docsutf8")   # imagine you're one directory above test dir
# all_keys = os.listdir(path+ "keys")   # imagine you're one directory above test dir
# print(len(all_files)," files \n",all_files, "\n", all_keys)  # won't necessarily be sorted

2000  files 
 ['642.txt', '1264.txt', '2007.txt', '1261.txt', '1673.txt', '1015.txt', '1198.txt', '967.txt', '39.txt', '1338.txt', '1898.txt', '1854.txt', '637.txt', '621.txt', '64.txt', '1984.txt', '1295.txt', '615.txt', '1641.txt', '2179.txt', '566.txt', '561.txt', '34.txt', '1385.txt', '287.txt', '1618.txt', '1185.txt', '1993.txt', '617.txt', '831.txt', '1914.txt', '631.txt', '981.txt', '220.txt', '1367.txt', '1719.txt', '927.txt', '36.txt', '854.txt', '707.txt', '2119.txt', '1356.txt', '108.txt', '111.txt', '1557.txt', '284.txt', '1266.txt', '989.txt', '783.txt', '1524.txt', '560.txt', '386.txt', '114.txt', '1967.txt', '1001.txt', '200.txt', '1526.txt', '1686.txt', '1604.txt', '2184.txt', '848.txt', '2093.txt', '922.txt', '1793.txt', '2160.txt', '1862.txt', '1761.txt', '2192.txt', '2096.txt', '1899.txt', '974.txt', '673.txt', '97.txt', '923.txt', '1634.txt', '1964.txt', '2120.txt', '838.txt', '241.txt', '1005.txt', '1991.txt', '1522.txt', '359.txt', '2198.txt', '1319.txt', '2048.tx

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import ast
############ Average p@k and mean avg P@k metric


def preproc_keys(key_list):
  kws = []
  key_list = ast.literal_eval(key_list)
  for kw in key_list:
    kws.append(processing(kw))
  return kws
def processing(t):
  return t.replace('\n','').replace('\t','').replace("'\\'",'').strip()

def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # print(p)
        if p in actual and p not in predicted[:i]:
            # print(p , 'in ', actual)
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    # print(score)
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [ ]:
incpec_data = pd.read_csv("/content/drive/MyDrive/PHD/article2023YA/data/incpec_data", sep ="\t")
incpec_data.head()

,Unnamed: 0,goldkeys,text
0,0,"['sampling density compensation', ' MRI image ...",Reconstruction of MR images from data acquired...
1,1,"['vanishing point estimation', ' automatic dri...",Estimation of the vanishing point for automati...
2,2,"['high-fidelity finite element model', ' kidne...",The creation of a high-fidelity finite element...
3,3,"['level set methods', ' topology-adaptive mode...",Topology-adaptive modeling of objects using su...
4,4,"['mission planning', ' regional surveillance',...",Mission planning for regional surveillance\n T...


In [ ]:
incpec_data['cleaned_text'] = incpec_data.text.apply(lambda x: processing(x))
incpec_data['goldkeys_cleaned'] = incpec_data.goldkeys.apply(lambda x: preproc_keys(x))

In [ ]:
df = incpec_data.loc[0:100]

# TF

In [ ]:
gold_kws = incpec_data.goldkeys_cleaned.tolist()[0:100]
docs = incpec_data.cleaned_text.tolist()[0:100]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1,1), stop_words = "english" )
vectors = vectorizer.fit_transform(docs)
##########################
index_value={i[1]:i[0] for i in vectorizer.vocabulary_.items()}

############################

tf_vectors = []  # all deoc vectors by tfidf
for row in vectors:
  tf_vectors.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

In [ ]:
tf_vectors[0]

{'reconstruction': 1,
 'mr': 2,
 'images': 4,
 'data': 4,
 'acquired': 3,
 'arbitrary': 2,
 'space': 2,
 'trajectory': 3,
 'using': 1,
 'image': 2,
 'weight': 8,
 'sampling': 1,
 'density': 1,
 'compensation': 1,
 'function': 1,
 'denoted': 1,
 'si': 5,
 'proposed': 1,
 'reconstruct': 2,
 'equation': 1,
 'established': 1,
 'criterion': 1,
 'iterative': 1,
 'scheme': 1,
 'developed': 1,
 'used': 2,
 'calculated': 1,
 'random': 1,
 'numerical': 1,
 'phantom': 1,
 'case': 1,
 'interleaved': 1,
 'spirals': 1,
 'vivo': 1,
 'experiment': 1,
 'respectively': 1,
 'addition': 1,
 'pipe': 2,
 'menon': 1,
 'mrm': 1,
 '1999': 1,
 '41': 1,
 '179': 1,
 '186': 1,
 'reconstructions': 1,
 'make': 1,
 'comparison': 1,
 'obtained': 2,
 'slightly': 1,
 'accurate': 1}

In [ ]:
doc_sorted_tfidfs =[]  # list of doc features each with tf weight
#sort each dict of a document
for dn in tf_vectors:
  newD = sorted(dn.items(), key=lambda x: x[1], reverse=True)
  newD = dict(newD)
  doc_sorted_tfidfs.append(newD)

In [ ]:
doc_sorted_tfidfs[0]

In [ ]:
gold_kws[0]

['sampling density compensation',
 'MRI image reconstruction',
 'arbitrary k-space trajectory',
 'spiral trajectory',
 'same-image weight',
 'random trajectory',
 'convolution function',
 'numerical phantom',
 'Nyquist sampling conditions',
 'iterative algorithm',
 'weighting function',
 'biomedical MRI',
 'convolution',
 'image reconstruction',
 'image sampling',
 'iterative methods',
 'medical image processing',
 '']

In [ ]:
# remove duplication from kws
# if kw has unigram with high freq set it in top N
all_kw = []
for i, doc_kw in enumeration(all_kw):
  topN = [] # ranked kw
  kw_dict ={}
  for kw in doc_kw:
    if set(kw.split()) & set(topN):
      if kw in kw_dict.keys():
        kw_dict[kw] = kw_dict[kw] + len(set(kw.split()) & set(topN))
      else:
        kw_dict[kw] = len(&)

# SpaCy

In [ ]:
!python -m spacy download en_core_web_lg

2023-07-14 12:53:01.081216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 12:53:02.679473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 947.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import spacy
# Load the English language model
nlp = spacy.load("en_core_web_lg")

In [ ]:
# Add the sentencizer component to the pipeline
nlp.add_pipe("sentencizer")

In [ ]:
def spacy_sentences(text):
  sents = []
  # Process the text
  doc = nlp(text)

  # Iterate over the sentences and print them
  for sent in doc.sents:
    sents.append(sent)
  return sents

def spacy_np(sents):
  nps =[]
  for sent_doc in sents:
    for np in sent_doc.noun_chunks:
      nps.append(np)
  return nps

def spacy_np_from_text(sents):
  nps =[]
  for sent_doc in sents:
    doc = nlp(sent_doc.text)
    for np in doc.noun_chunks:
      nps.append(np)
  return nps


def count_sents(df_sents):

  i= 0
  for se in df_sents:
    i = i + len(se)
  return i

In [ ]:
# df = incpec_data.loc[0:100]

In [ ]:
%%time
stanza_df['spacy_sents'] = stanza_df.cleaned_text.apply(lambda x: spacy_sentences(x))

CPU times: user 3.29 s, sys: 3.58 ms, total: 3.29 s
Wall time: 3.31 s


In [ ]:
%%time
stanza_df['spacy_nps'] = stanza_df.spacy_sents.apply(lambda x: spacy_np_from_text(x))

CPU times: user 6.41 s, sys: 12.2 ms, total: 6.42 s
Wall time: 6.49 s


In [ ]:
# stanza_df['spacy_kws'] = stanza_df.spacy_nps.apply(lambda x: ast.literal_eval(x))

In [ ]:
predicted = stanza_df.spacy_nps.tolist()

In [ ]:
predicted[0]

[Reconstruction,
 MR images,
 data,
 an arbitrary k-space trajectory,
 the same-image weight A sampling density compensation function,
 "same-image (SI) weight,
 MR images,
 the data,
 an arbitrary k-space trajectory,
 An equation,
 the SI weight,
 the SI criterion,
 an iterative scheme,
 the weight,
 The SI weight,
 images,
 the data,
 a random trajectory,
 a numerical phantom case,
 the data,
 interleaved spirals,
 an in vivo experiment,
 addition,
 Pipe and Menon's weight,
 MRM,
 the reconstructions,
 a comparison,
 The images,
 the SI weight,
 those,
 Pipe's weight]

### kwa

In [ ]:
def extract_spacy_kws(noun_phrases: List[str]) -> List[str]:

  keyphrases = {}
  for np in noun_phrases:
      if len(np) == 1:
          if np[0].pos_ in {'NOUN', 'PROPN', 'ADJ'}:
              temp.append(np.text.strip())
      elif 1 <len(np)< 6:
          if np[0].pos_ in {'DET'}:
              temp.append(np[1:].text)
          else:
              temp.append(np.text.strip())

  return keyphrases

In [ ]:
def clear_spacy_kws(list_kws):

  st_noun_phrases = []
  for nps in list_kws:
    temp = []
    for np in nps:
      if len(np) == 1:
        if np[0].pos_ in {'NOUN', 'PROPN', 'ADJ'}:
          temp.append(np.text.strip())
      elif 1 <len(np)< 6:
        if np[0].pos_ in {'DET'}:
          temp.append(np[1:].text)
        else:
          temp.append(np.text.strip())

    st_noun_phrases.append(temp)
  return st_noun_phrases

In [ ]:
sp_noun_phrases = clear_spacy_kws(predicted)

In [ ]:
sp_noun_phrases[61]

['Fresh tracks',
 'food processing',
 'code labels',
 'wireless terminals',
 'centralized database',
 'meat products',
 'customers',
 'Farmland Foods']

In [ ]:
mapk(gold_kws, sp_noun_phrases, 20)

0.12473543046466666

In [ ]:
gold_kws[61]

['food processing',
 'bar code labels',
 'wireless terminals',
 'Farmland Foods',
 'automatic data capture',
 'Intermec Technologies',
 'bar codes',
 'data acquisition',
 'food processing industry',
 'mobile computing',
 'production control',
 '']

## new DS

In [ ]:
stanza_df = pd.read_csv("/content/drive/MyDrive/PHD/article2023YA/data/spacy_stanza_100", sep ="\t")

In [ ]:
stanza_df.stanza_kws.tolist()[0]

'{\'MRM 1999\', \'the same - image weight\', \'data acquired on an arbitrary k- space trajectory using the same - image weight A sampling density compensation function denoted " same - image -LRB- SI -RRB- weight "\', "those obtained with Pipe \'s weight", \'A sampling density compensation function\', \'Reconstruction\', \'The images obtained with the SI weight\', \'images from the data calculated on a random trajectory in a numerical phantom case and from the data acquired on interleaved spirals in an in vivo experiment , respectively\', \'MR images from the data acquired on an arbitrary k- space trajectory\', \'the data acquired on an arbitrary k- space trajectory\', \'Reconstruction of MR images from data acquired on an arbitrary k- space trajectory using the same - image weight A sampling density compensation function denoted " same - image -LRB- SI -RRB- weight "\', \'a random trajectory in a numerical phantom case\', \'data\', "Pipe and Menon \'s weight -LRB- MRM 1999 ; 41:179-18

# get tree

In [ ]:
from nltk.tree import Tree
def extract_phrase(tree_str, label):
    phrases = []
    trees = Tree.fromstring(tree_str)
    for tree in trees:
        for subtree in tree.subtrees():
            if subtree.label() == label:
                t = subtree
                t = ' '.join(t.leaves())
                phrases.append(t)

    return phrases

# Stanza

In [ ]:
pip install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351311 sha256=932873db9be07b672fbdca9e5afedae68a87ea0005354374be290571f681a03f
  Stored in directory: /root/.cache/pip/wheels/ea/0b/64/114bc939d0083621aa41521e21be246c888260b8aa21e6c1ad
Successfully built emoji


In [ ]:
# pip install spacy_stanza

In [ ]:
import stanza
# import spacy_stanza

stanza.download("en")

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
nlp_stanza = stanza.Pipeline()

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
sent = "We use the uniqueness of each social media platform and the tools available to create a persona of your brand which then becomes part of your consumer's life"
sent = "Most of all, I enjoy working with travel partners to improve your team's Social Media savvy, content creation and digital marketing skills - and proving how actively participating in these areas can help grow your travel business!"
# %%time
sent = "I bought an apple. Apple is green and that costs 5 dollars"
stanza_nps(stanza_sents(sent))

{'5 dollars', 'Apple', 'I', 'an apple', 'that'}

In [ ]:
def stanza_sents(text):
  sents = []
  doc = nlp_stanza(text)

  for sent in doc.sentences:
    sents.append(sent.text)
  return sents

def stanza_nps(sentences):
  nps = set()
  for sent in sentences:
    doc = nlp_stanza(sent)
    sentence = doc.sentences[0]
    constituency_parse = sentence.constituency
    c = str(constituency_parse)
    np_list = extract_phrase(c, 'NP')
    nps.update(np_list)
  return nps


In [ ]:
df = incpec_data.loc[0:100]

In [ ]:
%%time
df['stanza_sents'] = df.cleaned_text.apply(lambda x: stanza_sents(x))

CPU times: user 16min 4s, sys: 2.94 s, total: 16min 7s
Wall time: 16min 20s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
%%time
df['stanza_nps'] = df.stanza_sents.apply(lambda x: stanza_nps(x))

CPU times: user 25min 5s, sys: 2.78 s, total: 25min 8s
Wall time: 25min 20s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
stanza_df.to_csv("/content/drive/MyDrive/PHD/article2023YA/data/spacy_stanza_100", sep='\t')

In [ ]:
st_nps = df['stanza_nps'].tolist()

In [ ]:
import spacy
# Load the English language model
nlp = spacy.load("en_core_web_sm")

In [ ]:
def clear_stanza_kws(list_kws):

  st_noun_phrases = []
  for nps in list_kws:
    temp = []
    for np in nps:
      tokens = np.split()
      if len(tokens) == 1:
        if nlp(tokens[0])[0].pos_ in {'NOUN', 'PROPN', 'ADJ'}:
          temp.append(np.strip())
      elif 1 <len(tokens)< 6:
        if tokens[0].lower() in {'a', 'an', 'the'}:
          temp.append(' '.join(tokens[1:]).strip())
        else:
          temp.append(np.strip())

    st_noun_phrases.append(temp)
  return st_noun_phrases

In [ ]:
stanza_df['stanza_kws'] = stanza_df.stanza_nps.apply(lambda x: ast.literal_eval(x))

In [ ]:
golds = stanza_df.goldkeys_cleaned.tolist()

In [ ]:
st_phrases = stanza_df.stanza_kws.tolist()

In [ ]:
st_phrases[0]

In [ ]:
st_noun_phrases = clear_stanza_kws(st_phrases)

In [ ]:
mapk(gold_kws, st_noun_phrases, 20)

0.08187798110428098

In [ ]:
st_noun_phrases[61]

['wireless terminals',
 'customers',
 'Farmland Foods',
 'food processing',
 'customers for Farmland Foods',
 'Fresh tracks',
 'meat products',
 'centralized database',
 'Bar code labels']

# spacy stanza

In [ ]:
pip install spacy_stanza

In [ ]:
import stanza
import spacy_stanza

stanza.download("en")
nlp = spacy_stanza.load_pipeline("en")

In [ ]:
doc = nlp("the president Barack Obama was born in Hawaii. He was elected president in 2008.")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type_)


the the DET det 
president president NOUN nsubj:pass 
Barack Barack PROPN appos PERSON
Obama Obama PROPN flat PERSON
was be AUX aux:pass 
born bear VERB root 
in in ADP case 
Hawaii Hawaii PROPN obl GPE
. . PUNCT punct 
He he PRON nsubj:pass 
was be AUX aux:pass 
elected elect VERB root 
president president NOUN xcomp 
in in ADP case 
2008 2008 NUM obl DATE
. . PUNCT punct 


In [ ]:
print([c for c in doc.noun_chunks])

[Barack]


# stanford nlp

In [ ]:
pip install stanfordnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
import stanfordnlp

MODELS_DIR = '.'
stanfordnlp.download('en', MODELS_DIR) # Download the English models
nlp_stanford = stanfordnlp.Pipeline(processors='tokenize,pos', models_dir=MODELS_DIR, treebank='en_ewt', use_gpu=True, pos_batch_size=3000) # Build the pipeline, specify part-of-speech processor's batch size

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Download location: ./en_ewt_models.zip


100%|██████████| 235M/235M [00:42<00:00, 5.59MB/s]



Download complete.  Models saved to: ./en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': './en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': './en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': './en_ewt_models/en_ewt.pretrain.pt', 'batch_size': 3000, 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [ ]:
sent = "I bought an apple that is green and that costs 5 dollars"

doc = nlp_stanford(sent) # Run the pipeline on input text
doc.sentences[0].print_tokens() # Look at the result
# print(*[f'text: {word.text+" "}\tlemma: {word.lemma}\tupos: {word.upos}\txpos: {word.xpos}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
sentence = doc.sentences
s = sentence[0]
s.print_dependencies
s.print_words

<bound method Sentence.print_words of <stanfordnlp.pipeline.doc.Sentence object at 0x7f5f2393d660>>

# Allen nlp

In [ ]:
pip install allennlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 458.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 96.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 29.5 MB/s eta 0:00:00
  Preparing metadat

In [ ]:
from allennlp.predictors.predictor import Predictor

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator12recordStreamERKNS_7DataPtrENS0_10CUDAStreamE
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
# import allennlp_models

In [ ]:
# pip uninstall -y dataclasses

In [ ]:
pip install allennlp-models

  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5569 sha256=50b64f02dcf4c5a6cd9e9dfb800abc4aa1c0839761abbfc9ef1af4679913b898
  Stored in directory: /root/.cache/pip/wheels/84/ff/26/d3cfbd971e96c5aa3737ecfced81628830d7359b55fbb8ca3b
Successfully built word2number


In [ ]:
import allennlp_models

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Output()

Output()

Output()

In [ ]:
def allen_sentences(text):
  sents = []
  # Process the text
  doc = nlp(text)

  # Iterate over the sentences and print them
  for sent in doc.sents:
    sents.append(sent.text)
  return sents

def allen_nps(sents):
  nps = set()
  for sent in sents:
    res = predictor.predict(sentence=sent)
    nps.update(extract_phrase(res['trees'], 'NP') )
  return nps

In [ ]:
df = incpec_data.loc[0:100]

In [ ]:
%%time
df['allen_sents'] = df.cleaned_text.apply(lambda x: allen_sentences(x))

CPU times: user 3.38 s, sys: 9.24 ms, total: 3.39 s
Wall time: 3.44 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
%%time
df['allen_nps'] = df.allen_sents.apply(lambda x: allen_nps(x))

CPU times: user 10min 29s, sys: 1.1 s, total: 10min 30s
Wall time: 10min 36s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# alen results
# df.to_csv("/content/drive/MyDrive/PHD/article2023YA/data/result_incpec_100", sep='\t')

In [ ]:
allen_df = pd.read_csv("/content/drive/MyDrive/PHD/article2023YA/data/result_incpec_100", sep='\t')

In [ ]:
allen_df['allen_kws'] = allen_df.allen_nps.apply(lambda x: ast.literal_eval(x))

In [ ]:
allen_df.allen_sents.tolist()[61]

"['Fresh tracks [food processing] Bar code labels and wireless terminals linked to a centralized database accurately track meat products from receiving to customers for Farmland Foods']"

In [ ]:
allen_kws= allen_df.allen_kws.tolist()


In [ ]:
def clear_allen_kws(list_kws):

  st_noun_phrases = []
  for nps in list_kws:
    temp = []
    for np in nps:
      tokens = np.split()
      if len(tokens) == 1:
        if nlp(tokens[0])[0].pos_ in {'NOUN', 'PROPN', 'ADJ'}:
          temp.append(np.strip())
      elif 1 <len(tokens)< 6:
        if tokens[0].lower() in {'a', 'an', 'the'}:
          temp.append(' '.join(tokens[1:]).strip())
        else:
          temp.append(np.strip())

    st_noun_phrases.append(temp)
  return st_noun_phrases

In [ ]:
allen_noun_phrases = clear_allen_kws(allen_kws)

In [ ]:
allen_kws[61]

{'Farmland Foods',
 'Fresh tracks',
 'Fresh tracks [ food processing ] Bar code labels and wireless terminals linked to a centralized database',
 'a centralized database',
 'customers',
 'meat products',
 'wireless terminals'}

In [ ]:
mapk(gold_kws, allen_noun_phrases, k=20)

0.07354793577655187

# Yake

In [ ]:
pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-f93u85cj
  Running command git clone --filter=blob:none --quiet https://github.com/LIAAD/yake /tmp/pip-req-build-f93u85cj
  Resolved https://github.com/LIAAD/yake to commit 374fc1c1c19eb080d5b6115cbb8d4a4324392e54
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00
  Created wheel for yake: filename=yake-0.4.8-py2.py3-none-any.whl size=62575 sha256=b5b367ba66a1952213eed01e21cde4645cef5a7759ed290ad87d6973a5e5781f
  Stored in directory: /tmp/pip-ephem-wheel-cache-lf89m6ta/wheels/10/9d/33/6a3358fd876c3d7c6c5c139d1496eb4b1618c7d0e15c375584
Successfully built yake


In [ ]:
import yake

In [ ]:
incpec_data.columns

Index(['Unnamed: 0', 'goldkeys', 'text', 'cleaned_text', 'goldkeys_cleaned'], dtype='object')

In [ ]:
gold_kws = incpec_data.goldkeys_cleaned.tolist()[0:100]
docs = incpec_data.cleaned_text.tolist()[0:100]
ranked_yake_kywords = []
for i, doc in enumerate(docs):
  # top = len(gold_kws[i])
  #print(top)
  kw_extractor = yake.KeywordExtractor(top=20, n=3)
  keywords = kw_extractor.extract_keywords(doc)

  #print("resulted ", len(keywords))
  temp = []
  for kw, v in keywords:
    temp.append(kw)
    #print(kw, ":", v)
  ranked_yake_kywords.append(temp)

In [ ]:
ranked_yake_kywords[61]

['Bar code labels',
 'wireless terminals linked',
 'centralized database accurately',
 'database accurately track',
 'accurately track meat',
 'track meat products',
 'Bar code',
 'Farmland Foods',
 'customers for Farmland',
 'food processing',
 'Fresh tracks',
 'code labels',
 'labels and wireless',
 'wireless terminals',
 'terminals linked',
 'centralized database',
 'database accurately',
 'meat products',
 'products from receiving',
 'receiving to customers']

In [ ]:
mapk(gold_kws, ranked_yake_kywords, k=20)

0.07573594695854174

In [ ]:
docs[61]

'Fresh tracks [food processing] Bar code labels and wireless terminals linked to a centralized database accurately track meat products from receiving to customers for Farmland Foods'

# Rake

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pip install rake-nltk

In [ ]:
gold_kws = incpec_data.goldkeys_cleaned.tolist()[0:100]
docs = incpec_data.cleaned_text.tolist()[0:100]

In [ ]:
for i,d in enumerate(docs):
  if len(d.split()) == 26:
    print(i,d)

61 Fresh tracks [food processing] Bar code labels and wireless terminals linked to a centralized database accurately track meat products from receiving to customers for Farmland Foods


In [ ]:
gold_kws[61]

['food processing',
 'bar code labels',
 'wireless terminals',
 'Farmland Foods',
 'automatic data capture',
 'Intermec Technologies',
 'bar codes',
 'data acquisition',
 'food processing industry',
 'mobile computing',
 'production control',
 '']

In [ ]:
%%time
from rake_nltk import Rake

ranked_Rake_kywords = []
for i, doc in enumerate(docs):
  top = len(gold_kws[i])
  # Uses stopwords for english from NLTK, and all puntuation characters by
  # default
  r = Rake(max_length=3, min_length=1)

  # Extraction given the text.
  r.extract_keywords_from_text(doc)

  # To get keyword phrases ranked highest to lowest.
  rr = r.get_ranked_phrases()

  # To get keyword phrases ranked highest to lowest with scores.
  #rrr= r.get_ranked_phrases_with_scores()
  #print(rrr)

  ranked_Rake_kywords.append(rr)

CPU times: user 75.7 ms, sys: 0 ns, total: 75.7 ms
Wall time: 79.2 ms


In [ ]:
mapk(gold_kws, ranked_Rake_kywords, k=20)

0.07944403249126637

In [ ]:
ranked_Rake_kywords[61]

['wireless terminals linked',
 'bar code labels',
 'fresh tracks',
 'food processing',
 'farmland foods',
 'receiving',
 'customers']